In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# scaler 
from sklearn.preprocessing import PowerTransformer, MinMaxScaler, StandardScaler,QuantileTransformer
#
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

Import Dataset

In [ ]:
df = pd.read_csv("BankRecords.csv")

### EDA

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

Categorical Variables Unique Values

In [ ]:
df["Education"].unique()

In [ ]:
df["CD Account"].unique()

In [ ]:
df["Online Banking"].unique()

In [ ]:
df["Securities Account"].unique()

In [ ]:
df["Securities Account"].unique()

In [ ]:
df["Personal Loan"].unique()

Encoding Categorical Variables 

In [ ]:
education_encoder = {
  "Diploma" : 1,
  "Degree" : 2,
  "Masters" : 3
}

df["Education"] = df["Education"].map(education_encoder)

In [ ]:
binary_columns = ["Personal Loan", "Securities Account", "CD Account", "Online Banking", "CreditCard"]

df[binary_columns] = df[binary_columns].replace({"Yes": 1, "No": 0})

Distribution of age variable

In [ ]:
# Set the style of seaborn
sns.set_style("whitegrid")

# Plot histograms for numerical variables
sns.histplot(data=df, x='Age', bins=10, kde=True)

In [ ]:
sns.countplot(data=df, x='Education')

In [ ]:
sns.countplot(data=df, x="Family")
plt.xlabel('Family Size')
plt.ylabel('Count')
plt.title('Count of Family Sizes')
plt.show()


In [ ]:
sns.boxplot(data=df, y='Experience(Years)')

In [ ]:
sns.boxplot(data=df, x='Education', y="Income(Thousands's)")

In [ ]:
df.hist(figsize=(12,12))
plt.show()

In [ ]:
sns.pairplot(df[["Age", "Experience(Years)","Income(Thousands's)", "Credit Score"]])

In [ ]:
sns.histplot(data=df, x='Experience(Years)', bins=10, kde=True)

plt.xlabel('Experience (Years)')
plt.ylabel('Frequency')
plt.title('Distribution of Experience')

plt.show()

Matrix Correlation 

In [ ]:
correlation_matrix = df.corr()
  
# Visualize the correlation matrix using a heatmap
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", square=True)

Median Experience by Age

In [46]:
median_exp_by_age = df.groupby("Age")["Experience(Years)"].median()


print("Median experience by age:")
print(median_exp_by_age)

Median experience by age:
Age
23    -1.5
24    -1.0
25     0.0
26     1.0
27     2.0
28     3.0
29     4.0
30     5.0
31     5.0
32     7.0
33     8.0
34     9.0
35    10.0
36    11.0
37    12.0
38    13.0
39    14.0
40    15.0
41    16.0
42    17.0
43    18.0
44    19.0
45    20.0
46    21.0
47    22.0
48    23.0
49    24.0
50    25.0
51    26.0
52    27.0
53    28.0
54    29.0
55    30.0
56    31.0
57    32.0
58    33.0
59    34.0
60    35.0
61    36.0
62    37.0
63    38.0
64    39.0
65    40.0
66    41.0
67    42.0
Name: Experience(Years), dtype: float64


Number of records that experiance column has negative value

In [ ]:
filtered_df = df[(df["Experience(Years)"] <= 0) & (df["Income(Thousands's)"] > 0)]

age_group_counts = filtered_df.groupby("Age").size()

print("Number of records for each age group where experience is smaller than 0:")
print(age_group_counts)

Total number of records for each age

In [ ]:
age_group_counts = df.groupby("Age").size()

print("Number of records for each age group:")
print(age_group_counts)
